# Flash Evaluation on DARPA E3 Fivedirections Dataset: 

This notebook is specifically designed for the evaluation of Flash on the DARPA E3 Fivedirections dataset. Notably, the Fivedirections dataset is characterized as a node-level dataset. In our analysis, Flash is configured to operate in a node-level setting to aptly assess this dataset. A key aspect to note is that the Fivedirections dataset lacks certain essential node attributes for specific node types. This limitation means that Flash cannot be operated in a decoupled mode with offline GNN embeddings for this dataset. Consequently, we employ an online GNN coupled with word2vec semantic embeddings to achieve effective evaluation results for this dataset.

## Dataset Access: 
- Access the Fivedirections dataset via the following link: [Fivedirections Dataset](https://drive.google.com/drive/folders/1QlbUFWAGq3Hpl8wVdzOdIoZLFxkII4EK).
- The dataset files will be downloaded automatically by the script.

## Data Parsing and Execution:
- The script is designed to automatically parse the downloaded data files.
- Execute all cells within this notebook to obtain the evaluation results.

## Model Training and Execution Flexibility:
- The notebook is configured to use pre-trained model weights by default.
- It also provides the option to set parameters for independently training Graph Neural Networks (GNNs) and word2vec models.
- These newly trained models can then be utilized for a comprehensive evaluation of the dataset.

Adhere to these steps for a detailed and effective analysis of the Fivedirections dataset using Flash.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import os
import torch.nn.functional as F
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import multiprocessing
import csv

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

import subprocess
gpu_mem = {int(x.split(',')[0]): int(x.split(',')[1]) for x in subprocess.check_output(
    ["nvidia-smi", "--query-gpu=index,memory.free", "--format=csv,noheader,nounits"], 
    encoding='utf-8').strip().split('\n')}
best_gpu = max(gpu_mem.items(), key=lambda x: x[1])[0]
device = torch.device(f'cuda:{best_gpu}' if torch.cuda.is_available() else 'cpu')
print(device)

%matplotlib inline

cuda:0


In [2]:
# import gdown
# urls = ["https://drive.google.com/file/d/1BeP80zUUmm4eZl0UuU43PsKNkl_xgskj/view?usp=drive_link"]
# for url in urls:
#     gdown.download(url, quiet=False, use_cookies=False, fuzzy=True)

In [3]:
Train = True

In [4]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os

In [5]:
import re

def extract_uuid(line):
    pattern_uuid = re.compile(r'uuid\":\"(.*?)\"')
    return pattern_uuid.findall(line)

def extract_subject_type(line):
    pattern_type = re.compile(r'type\":\"(.*?)\"')
    return pattern_type.findall(line)

def show(file_path):
    print(f"Processing {file_path}")

def extract_edge_info(line):
    pattern_src = re.compile(r'subject\":{\"com.bbn.tc.schema.avro.cdm18.UUID\":\"(.*?)\"}')
    pattern_dst1 = re.compile(r'predicateObject\":{\"com.bbn.tc.schema.avro.cdm18.UUID\":\"(.*?)\"}')
    pattern_dst2 = re.compile(r'predicateObject2\":{\"com.bbn.tc.schema.avro.cdm18.UUID\":\"(.*?)\"}')
    pattern_type = re.compile(r'type\":\"(.*?)\"')
    pattern_time = re.compile(r'timestampNanos\":(.*?),')

    edge_type = extract_subject_type(line)[0]
    timestamp = pattern_time.findall(line)[0]
    src_id = pattern_src.findall(line)

    if len(src_id) == 0:
        return None, None, None, None, None

    src_id = src_id[0]
    dst_id1 = pattern_dst1.findall(line)
    dst_id2 = pattern_dst2.findall(line)

    if len(dst_id1) > 0 and dst_id1[0] != 'null':
        dst_id1 = dst_id1[0]
    else:
        dst_id1 = None

    if len(dst_id2) > 0 and dst_id2[0] != 'null':
        dst_id2 = dst_id2[0]
    else:
        dst_id2 = None

    return src_id, edge_type, timestamp, dst_id1, dst_id2

def process_data(file_path):
    id_nodetype_map = {}
    notice_num = 1000000
    for i in range(100):
        now_path = file_path + '.' + str(i)
        if i == 0:
            now_path = file_path
        if not os.path.exists(now_path):
            break

        with open(now_path, 'r') as f:
            show(now_path)
            cnt = 0
            for line in f:
                cnt += 1
                if cnt % notice_num == 0:
                    print(cnt)

                if 'com.bbn.tc.schema.avro.cdm18.Event' in line or 'com.bbn.tc.schema.avro.cdm18.Host' in line:
                    continue

                if 'com.bbn.tc.schema.avro.cdm18.TimeMarker' in line or 'com.bbn.tc.schema.avro.cdm18.StartMarker' in line:
                    continue

                if 'com.bbn.tc.schema.avro.cdm18.UnitDependency' in line or 'com.bbn.tc.schema.avro.cdm18.EndMarker' in line:
                    continue

                uuid = extract_uuid(line)[0]
                subject_type = extract_subject_type(line)

                if len(subject_type) < 1:
                    if 'com.bbn.tc.schema.avro.cdm18.MemoryObject' in line:
                        id_nodetype_map[uuid] = 'MemoryObject'
                        continue
                    if 'com.bbn.tc.schema.avro.cdm18.NetFlowObject' in line:
                        id_nodetype_map[uuid] = 'NetFlowObject'
                        continue
                    if 'com.bbn.tc.schema.avro.cdm18.UnnamedPipeObject' in line:
                        id_nodetype_map[uuid] = 'UnnamedPipeObject'
                        continue

                id_nodetype_map[uuid] = subject_type[0]

    return id_nodetype_map

def process_edges(file_path, id_nodetype_map):
    notice_num = 1000000
    not_in_cnt = 0

    for i in range(100):
        now_path = file_path + '.' + str(i)
        if i == 0:
            now_path = file_path
        if not os.path.exists(now_path):
            break

        with open(now_path, 'r') as f, open(now_path+'.txt', 'w') as fw:
            cnt = 0
            for line in f:
                cnt += 1
                if cnt % notice_num == 0:
                    print(cnt)

                if 'com.bbn.tc.schema.avro.cdm18.Event' in line:
                    src_id, edge_type, timestamp, dst_id1, dst_id2 = extract_edge_info(line)

                    if src_id is None or src_id not in id_nodetype_map:
                        not_in_cnt += 1
                        continue

                    src_type = id_nodetype_map[src_id]

                    if dst_id1 is not None and dst_id1 in id_nodetype_map:
                        dst_type1 = id_nodetype_map[dst_id1]
                        this_edge1 = f"{src_id}\t{src_type}\t{dst_id1}\t{dst_type1}\t{edge_type}\t{timestamp}\n"
                        fw.write(this_edge1)

                    if dst_id2 is not None and dst_id2 in id_nodetype_map:
                        dst_type2 = id_nodetype_map[dst_id2]
                        this_edge2 = f"{src_id}\t{src_type}\t{dst_id2}\t{dst_type2}\t{edge_type}\t{timestamp}\n"
                        fw.write(this_edge2)

def run_data_processing():
    os.system('tar -zxvf ta1-fivedirections-e3-official-2.json.tar.gz')

    path_list = ['ta1-fivedirections-e3-official-2.json']

    for path in path_list:
        id_nodetype_map = process_data(path)
        process_edges(path, id_nodetype_map)

    os.system('cp ta1-fivedirections-e3-official-2.json.txt fivedirections_train.txt')
    os.system('cp ta1-fivedirections-e3-official-2.json.23.txt fivedirections_test.txt')

In [6]:
# run_data_processing()

In [7]:
def prepare_graph(df):
    nodes = {}
    labels = {}
    edges = []
    
    dummies = {'SUBJECT_PROCESS': 0, 'FILE_OBJECT_CHAR': 1, 'VALUE_TYPE_SRC': 2, 'SRCSINK_DATABASE': 3,
               'FILE_OBJECT_UNIX_SOCKET': 4,'FILE_OBJECT_BLOCK': 5, 'NetFlowObject': 6, 
               'SRCSINK_PROCESS_MANAGEMENT': 7, 'SUBJECT_THREAD': 8}

    for i in range(len(df)):
        x = df.iloc[i]
        if x['object'] in dummies:
            action = x["action"]

            actorid = x["actorID"]
            if not (actorid in nodes):
                nodes[actorid] =  []
            if x['exec'] != '':
                nodes[actorid].append(x['exec'])
            nodes[actorid].append(action)
            if x['path'] != '':
                nodes[actorid].append(x['path'])
            labels[actorid] = dummies[x['actor_type']]

            objectid = x["objectID"]
            if not (objectid in nodes):
                nodes[objectid] =  []
            if x['exec'] != '':
                nodes[objectid].append(x['exec'])
            nodes[objectid].append(action)
            if x['path'] != '':
                 nodes[objectid].append(x['path'])
            labels[objectid] = dummies[x['object']]

            edges.append(( actorid, objectid, action ))

    features = []
    feat_labels = []
    edge_index = [[],[]]
    index  = {}
    mapp = []

    for k,v in nodes.items():
      features.append(v)
      feat_labels.append(labels[k])
      index[k] = len(features) - 1
      mapp.append(k)

    for x in edges:
        src = index[x[0]]
        dst = index[x[1]]
        
        edge_index[0].append(src)
        edge_index[1].append(dst)             

    return features,feat_labels,edge_index,mapp,index

dummies = {'SUBJECT_PROCESS': 0, 'FILE_OBJECT_CHAR': 1, 'VALUE_TYPE_SRC': 2, 'SRCSINK_DATABASE': 3,
               'FILE_OBJECT_UNIX_SOCKET': 4,'FILE_OBJECT_BLOCK': 5, 'NetFlowObject': 6, 
               'SRCSINK_PROCESS_MANAGEMENT': 7, 'SUBJECT_THREAD': 8}

from collections import defaultdict


def add_csv_edges_efficient(original_features, original_labels, original_edge_index, original_mapp, original_mapidx, csv_path, thres=450): 
    csv_edges = []
    csv_nodes = set()
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            src_id, dst_id = row[0], row[1]
            csv_edges.append((src_id, dst_id))
            csv_nodes.add(src_id)
            csv_nodes.add(dst_id)
    
    extended_features = original_features.copy()
    extended_labels = original_labels.copy()
    extended_edge_index = [original_edge_index[0].copy(), original_edge_index[1].copy()]
    extended_mapp = original_mapp.copy()
    extended_mapidx = original_mapidx.copy()

    degrees = defaultdict(int)
    for src_idx in extended_edge_index[0]:
        degrees[src_idx] += 1
    for dst_idx in extended_edge_index[1]:
        degrees[dst_idx] += 1
    
    all_degrees = list(degrees.values())
    all_degrees.sort(reverse=True)
    print(all_degrees[:3000])

    count1, count2, count3 = 0, 0, 0
    for src_id, dst_id in csv_edges:
        if src_id in extended_mapidx and dst_id in extended_mapidx:
            src_idx = extended_mapidx[src_id]
            dst_idx = extended_mapidx[dst_id]
            if degrees[src_idx] <= thres and degrees[dst_idx] <= thres:
                extended_edge_index[0].append(src_idx)
                extended_edge_index[1].append(dst_idx)
                count1 += 1
            else:
                count2 += 1
        else:
            count3 += 1
    print(f'Sucessfully add edges: {count1}\tPrune edges: {count2}\tFail to add edges: {count3}')
    
    return extended_features, extended_labels, extended_edge_index, extended_mapp, extended_mapidx

In [8]:
import random 

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [9]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GATConv
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self,in_channel,out_channel):
        super().__init__()
        self.conv1 = SAGEConv(in_channel, 32, normalize=True)
        self.conv2 = SAGEConv(32, out_channel, normalize=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        return x

In [10]:
def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [11]:
from gensim.models.callbacks import CallbackAny2Vec
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm
import time

class EpochSaver(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('word2vec_five_E3.model')
        self.epoch += 1

In [12]:
class EpochLogger(CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [13]:
logger = EpochLogger()
saver = EpochSaver()

In [14]:
def add_attributes(d,p):
    
    f = open(p)
    data = [json.loads(x) for x in f if "EVENT" in x]

    info = []
    for x in data:
        try:
            action = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['type']
        except:
            action = ''
        try:
            actor = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['subject']['com.bbn.tc.schema.avro.cdm18.UUID']
        except:
            actor = ''
        try:
            obj = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject']['com.bbn.tc.schema.avro.cdm18.UUID']
        except:
            obj = ''
        try:
            timestamp = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['timestampNanos']
        except:
            timestamp = ''
        try:
            cmd = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['properties']['map']['exec']
        except:
            cmd = ''
        try:
            path = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObjectPath']['string']
        except:
            path = ''
        try:
            path2 = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject2Path']['string']
        except:
            path2 = ''
        try:
            obj2 = x['datum']['com.bbn.tc.schema.avro.cdm18.Event']['predicateObject2']['com.bbn.tc.schema.avro.cdm18.UUID']
            info.append({'actorID':actor,'objectID':obj2,'action':action,'timestamp':timestamp,'exec':cmd, 'path':path2})
        except:
            pass

        info.append({'actorID':actor,'objectID':obj,'action':action,'timestamp':timestamp,'exec':cmd, 'path':path})

    rdf = pd.DataFrame.from_records(info).astype(str)
    d = d.astype(str)

    return d.merge(rdf,how='inner',on=['actorID','objectID','action','timestamp']).drop_duplicates()

In [15]:
if Train:
    f = open("../fivedirections_train.txt")
    data = f.read().split('\n')
    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df = df.dropna()
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = add_attributes(df,"../ta1-fivedirections-e3-official-2.json")
    phrases,labels,edges,mapp,mapidx = prepare_graph(df)
    csv_path = "../../flash-add-edge/fivedirections_train.csv"
    phrases, labels, edges, mapp, mapidx = add_csv_edges_efficient(
        phrases, labels, edges, mapp, mapidx, csv_path
    )

[250678, 249140, 131930, 115200, 82181, 40167, 34263, 32537, 31683, 30792, 20657, 19939, 18628, 16520, 15794, 15747, 15390, 14611, 14131, 14063, 12045, 12019, 11646, 9999, 9988, 9926, 9917, 9608, 9551, 9544, 9468, 9408, 9395, 9385, 9382, 9356, 9345, 9338, 9333, 9332, 9330, 9328, 9318, 9311, 9309, 9304, 9286, 8402, 7538, 7394, 7331, 7287, 7205, 7199, 6951, 6943, 6872, 5963, 5439, 5434, 5219, 5069, 5011, 5003, 4997, 4965, 4952, 4811, 4791, 4478, 4038, 4034, 3972, 3971, 3971, 3970, 3968, 3967, 3966, 3966, 3964, 3964, 3963, 3961, 3961, 3961, 3961, 3959, 3959, 3958, 3957, 3955, 3730, 3631, 3576, 3554, 3540, 3533, 3473, 3426, 3284, 3273, 3245, 3245, 3218, 3164, 3057, 3047, 2951, 2818, 2790, 2789, 2776, 2751, 2645, 2640, 2512, 2492, 2449, 2315, 2303, 2236, 2178, 2149, 2120, 2108, 2085, 2027, 1984, 1873, 1789, 1746, 1731, 1619, 1554, 1514, 1470, 1465, 1413, 1383, 1375, 1363, 1362, 1327, 1324, 1323, 1309, 1290, 1260, 1237, 1235, 1215, 1214, 1206, 1205, 1204, 1198, 1181, 1181, 1176, 1174, 1165, 

In [ ]:
from sklearn.utils import class_weight
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN(30,9).to(device)
# model = GCN(10,9).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [17]:
# if Train:
#     word2vec = Word2Vec(sentences=phrases, vector_size=30, window=5, min_count=1, workers=8,epochs=300,callbacks=[saver,logger])

In [18]:
# if Train:
#     word2vec = Word2Vec(sentences=phrases, vector_size=10, window=5, min_count=1, workers=8,epochs=300,callbacks=[saver,logger])

In [19]:
import math
import torch
import numpy as np
from gensim.models import Word2Vec

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000):
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]


def infer(document):
    word_embeddings = [w2vmodel.wv[word] for word in document if word in  w2vmodel.wv]
    
    if not word_embeddings:
        return np.zeros(30)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    return np.mean(output_embedding, axis=0)

encoder = PositionalEncoder(30)
# encoder = PositionalEncoder(10)
# w2vmodel = Word2Vec.load("trained_weights/fivedirections/word2vec_five_E3.model")
w2vmodel = Word2Vec.load("../word2vec_five_E3.model")

In [20]:
from torch_geometric import utils

if Train:
    l = np.array(labels)
    ngram_class_weights = [
        1.0573,   
        1.0865,  
        1.1827,  
        1.0000,  
        0.5000,  
        0.5846,   
        1.5000,    
        1.0000,    
        0.9151,      
    ]
    class_weights = torch.tensor(ngram_class_weights,dtype=torch.float).to(device)
    criterion = CrossEntropyLoss(weight=class_weights,reduction='mean')

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes)
    mask = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)
    
    losses = []

    for m_n in range(13):
        model.load_state_dict(torch.load(f'../../trained_weights/fivedirections/lword2vec_gnn_five{m_n}_E3.pth'))
        loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
        total_loss = 0
        for subg in loader:
            model.train()
            optimizer.zero_grad() 
            out = model(subg.x, subg.edge_index) 
            loss = criterion(out, subg.y) 
            loss.backward() 
            optimizer.step()      
            total_loss += loss.item() * subg.batch_size
        losses.append(total_loss / mask.sum().item())
        print(total_loss / mask.sum().item())

        loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
        for subg in loader:
            model.eval()
            out = model(subg.x, subg.edge_index)

            sorted, indices = out.sort(dim=1,descending=True)
            conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
            conf = (conf - conf.min()) / conf.max()

            pred = indices[:,0]
            cond = (pred == subg.y) | (conf >= 0.9)
            subg.n_id = subg.n_id.to(device)
            mask[subg.n_id[cond]] = False

        torch.save(model.state_dict(), f'lword2vec_gnn_five{m_n}_E3.pth')
        print(f'Model# {m_n}. {mask.sum().item()} nodes still misclassified \n')

1.5645268665469447
Model# 0. 1781 nodes still misclassified 

1.716921329498291
Model# 1. 1769 nodes still misclassified 

1.7119152545928955
Model# 2. 1769 nodes still misclassified 

1.6803090572357178
Model# 3. 1769 nodes still misclassified 

1.6723934412002563
Model# 4. 1744 nodes still misclassified 

1.651169776916504
Model# 5. 1742 nodes still misclassified 

1.627303123474121
Model# 6. 1708 nodes still misclassified 

1.6450992822647095
Model# 7. 1624 nodes still misclassified 

1.6170694828033447
Model# 8. 1571 nodes still misclassified 

1.5851026773452759
Model# 9. 1567 nodes still misclassified 

1.585153341293335
Model# 10. 1557 nodes still misclassified 

1.5877386331558228
Model# 11. 1554 nodes still misclassified 

1.5870611667633057
Model# 12. 1312 nodes still misclassified 



In [21]:
from itertools import compress
from torch_geometric import utils

def Get_Adjacent(ids, mapp, edges, hops):
    if hops == 0:
        return set()
    
    neighbors = set()
    for edge in zip(edges[0], edges[1]):
        if any(mapp[node] in ids for node in edge):
            neighbors.update(mapp[node] for node in edge)

    if hops > 1:
        neighbors = neighbors.union(Get_Adjacent(neighbors, mapp, edges, hops - 1))
    
    return neighbors

def calculate_metrics(TP, FP, FN, TN):
    FPR = FP / (FP + TN) if FP + TN > 0 else 0
    TPR = TP / (TP + FN) if TP + FN > 0 else 0

    prec = TP / (TP + FP) if TP + FP > 0 else 0
    rec = TP / (TP + FN) if TP + FN > 0 else 0
    fscore = (2 * prec * rec) / (prec + rec) if prec + rec > 0 else 0

    return prec, rec, fscore, FPR, TPR

def helper(MP, all_pids, GP, edges, mapp):
    TP = MP.intersection(GP)
    FP = MP - GP
    FN = GP - MP
    TN = all_pids - (GP | MP)

    two_hop_gp = Get_Adjacent(GP, mapp, edges, 2)
    two_hop_tp = Get_Adjacent(TP, mapp, edges, 2)
    FPL = FP - two_hop_gp
    TPL = TP.union(FN.intersection(two_hop_tp))
    FN = FN - two_hop_tp

    TP, FP, FN, TN = len(TPL), len(FPL), len(FN), len(TN)

    prec, rec, fscore, FPR, TPR = calculate_metrics(TP, FP, FN, TN)
    print(f"True Positives: {TP}, True Negatives: {TN}, False Positives: {FP}, False Negatives: {FN}")
    print(f"Precision: {round(prec, 2)}, Recall: {round(rec, 2)}, Fscore: {round(fscore, 2)}")
    
    return TPL, FPL

In [22]:
f = open("../fivedirections_test.txt")
data = f.read().split('\n')
data = [line.split('\t') for line in data]
df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
df = df.dropna()
df.sort_values(by='timestamp', ascending=True,inplace=True)

In [23]:
df = add_attributes(df,"../ta1-fivedirections-e3-official-2.json.23")

In [24]:
csv_path = "../../flash-add-edge/fivedirections_test.csv"

data = df
phrases,labels,edges,mapp,mapidx = prepare_graph(data)
# phrases, labels, edges, mapp, mapidx = add_csv_edges_efficient(
#     phrases, labels, edges, mapp, mapidx, csv_path
# )
nodes = [infer(x) for x in phrases]
nodes = np.array(nodes)  

In [25]:
with open("../../data_files/fivedirections.json", "r") as json_file:
    GT_mal = json.load(json_file)
    
GT_mal = set([x for x in GT_mal if x in mapp])

all_ids = list(data['actorID']) + list(data['objectID'])
all_ids = set(all_ids)

In [ ]:
graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
graph.n_id = torch.arange(graph.num_nodes)
flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)
for m_n in range(13):
  # model.load_state_dict(torch.load(f'../../trained_weights/fivedirections/lword2vec_gnn_five{m_n}_E3.pth'))
  model.load_state_dict(torch.load(f'lword2vec_gnn_five{m_n}_E3.pth'))
  loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000)    
  for subg in loader:
      model.eval()
      out = model(subg.x, subg.edge_index)

      sorted, indices = out.sort(dim=1,descending=True)
      conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
      conf = (conf - conf.min()) / conf.max()
    
      pred = indices[:,0]
      cond = (pred == subg.y) & (conf > 0.94)
      subg.n_id = subg.n_id.to(device)
      flag[subg.n_id[cond]] = torch.logical_and(flag[subg.n_id[cond]], torch.tensor([False]*len(flag[subg.n_id[cond]]), dtype=torch.bool).to(device))

index = utils.mask_to_index(flag).tolist()
ids = set([mapp[x] for x in index])
alerts = helper(set(ids),set(all_ids),GT_mal,edges,mapp) 

True Positives: 364, True Negatives: 359084, False Positives: 81, False Negatives: 61
Precision: 0.82, Recall: 0.86, Fscore: 0.84


In [27]:
def traverse(ids, mapping, edges, hops, visited=None):
    if hops == 0:
        return set()

    if visited is None:
        visited = set()

    neighbors = set()
    for src, dst in zip(edges[0], edges[1]):
        src_mapped, dst_mapped = mapping[src], mapping[dst]

        if (src_mapped in ids and dst_mapped not in visited) or \
           (dst_mapped in ids and src_mapped not in visited):
            neighbors.add(src_mapped)
            neighbors.add(dst_mapped)

        visited.add(src_mapped)
        visited.add(dst_mapped)

    neighbors.difference_update(ids) 
    return ids.union(traverse(neighbors, mapping, edges, hops - 1, visited))

def load_data(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def find_connected_alerts(start_alert, mapping, edges, depth, remaining_alerts):
    connected_path = traverse({start_alert}, mapping, edges, depth)
    return connected_path.intersection(remaining_alerts)

def generate_incident_graphs(alerts, edges, mapping, depth):
    incident_graphs = []
    remaining_alerts = set(alerts)

    while remaining_alerts:
        alert = remaining_alerts.pop()
        connected_alerts = find_connected_alerts(alert, mapping, edges, depth, remaining_alerts)

        if len(connected_alerts) > 1:
            incident_graphs.append(connected_alerts)
            remaining_alerts -= connected_alerts

    return incident_graphs